### Итоговая работа
### 12448/3 ИНЖДАН МГТУ им. Н.Э. Баумана
### Инженер данных (Data engineer)
### Сайфуллин Руслан Рафитович

## КЛАССИФИКАЦИЯ ГОСКОНТРАКТОВ ПО ОБЪЕКТАМ ЗАКУПКИ

#### Задача: необходимо на основе данных с ftp.zakupki.gov.ru научиться определять группу, к которой относится контракт с кодом ОКПД-2 41, 42, 43, 71.1.

Группы могут быть следующими:

1.	Строительно-монтажные работы (СМР)
2.	Проектно-изыскательские работы (ПИР)
3.	Строительный надзор
4.	Подключение коммуникаций
5.	Прочее.

По ОКПД-2 контракты в общем случае должны разделяться так:
- Строительно-монтажные работы (СМР) - 41, 42, 43(кроме нижеперечисленных)
- Проектно-изыскательские работы (ПИР) - 41.1, 71.1
- Подключение коммуникаций - 43.22
- Строительный надзор – четкой группы нет.


Проблема: Далеко не всегда контракты указываются с нужным кодом, поэтому есть проблема как такие контракты "отловить" и определить в нужную группу.

Поэтому задача предполагает классификацию контрактов на основе объекта закупки, который сформулирован естественным языком. Также предполагаем, что могут иметь значение цена контракта и его длительность.
На основе этого на входе данные о контрактах. На выходе необходимо получить группу для каждого контракта.

Иногда контракт может относиться одновременно в несколько групп.

В приложении ниже пример нескольких контрактов, у которых неверно проставлен ОКПД-2.

## Решаем задачу классификации

In [1]:
import pandas as pd
import numpy as np

In [2]:
columns = ['object_name', 'object_code', 'cost', 'contract_execution_days']
dtypes = {'object_name': str,
          'object_code': str, 
          'cost':lambda x: pd.to_numeric(x, errors="coerce"), 
          'contract_execution_days': lambda x: pd.to_numeric(x, errors="coerce"),
          'cluster': int,
          'group': int}


In [3]:
df = pd.read_csv('datasets/dataset_cluster.csv', sep=';', low_memory=False, on_bad_lines='skip')
df.shape

(900153, 6)

In [4]:
df.columns

Index(['object_name', 'object_code', 'cost', 'contract_execution_days',
       'cluster', 'group'],
      dtype='object')

### Классификация контрактов по разметке полученых с помощью кластеризации

In [5]:
#X = df[['object_name', 'cost', 'contract_execution_days']]
X = df['object_name']
y = df['group']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape)

(648109,) (162028,)


In [7]:
from sklearn.pipeline import Pipeline
# pipeline позволяет объединить в один блок трансформер и модель, что упрощает написание кода и улучшает его читаемость
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer преобразует тексты в числовые вектора, отражающие важность использования каждого слова из некоторого набора слов (количество слов набора определяет размерность вектора) в каждом тексте
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
# линейный классификатор и классификатор методом ближайших соседей
from sklearn import metrics
# набор метрик для оценки качества модели
from sklearn.model_selection import GridSearchCV
# модуль поиска по сетке параметров

from sklearn.metrics import confusion_matrix

Сначала создадим 2 классификатора (чтобы можно было в дальнейшем сравнить качество получившихся моделей) и обучим их на тестовых данных:

In [8]:
sgd_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('sgd_clf', SGDClassifier(random_state=42))])
sgd_ppl_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('sgd_clf', SGDClassifier(random_state=42))])

In [9]:
knb_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])
knb_ppl_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('knb_clf', KNeighborsClassifier(n_neighbors=10))])

In [10]:
predicted_sgd = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     72494
           1       1.00      1.00      1.00     48403
           2       1.00      1.00      1.00     23010
           3       1.00      1.00      1.00     18121

    accuracy                           1.00    162028
   macro avg       1.00      1.00      1.00    162028
weighted avg       1.00      1.00      1.00    162028



In [11]:
confusion_matrix(predicted_sgd, y_test)

array([[72494,     0,     0,     0],
       [    0, 48403,     0,     0],
       [    0,     0, 23010,     0],
       [    0,     0,     0, 18121]], dtype=int64)

In [12]:
predicted_sgd = knb_ppl_clf.predict(X_test)
print(metrics.classification_report(predicted_sgd, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     72494
           1       1.00      1.00      1.00     48403
           2       1.00      1.00      1.00     23010
           3       1.00      1.00      1.00     18121

    accuracy                           1.00    162028
   macro avg       1.00      1.00      1.00    162028
weighted avg       1.00      1.00      1.00    162028



In [13]:
confusion_matrix(predicted_sgd, y_test)

array([[72494,     0,     0,     0],
       [    0, 48403,     0,     0],
       [    0,     0, 23010,     0],
       [    0,     0,     0, 18121]], dtype=int64)